In [ ]:
%matplotlib inline

# Tema: Redes Neuronales Feed-forward

Al igual que para la notebook de la clase 5 de SVM, utilizaremos el dataset "empty.all.csv" que contiene como clase positiva, 900 artículos de Wikipedia en inglés que presentan la falla "Empty Section" y como clase negativa, contiene 900 artículos destacados. El mismo se encuentra en el subdirectorio "miscelaneos" del repositorio Github. Los datos se cargan como un DataFrame mediante un método de la biblioteca seaborn. A tal fin es necesario copiar el dataset en el home local de seaborn. Por defecto usa ~/seaborn-data/, en Windows: "C:\Users\Nbre_Usuario\seaborn-data".

## Ejemplos

In [ ]:
import seaborn as sns
empty = sns.load_dataset('empty.all',cache=True)
empty.head()

In [ ]:
list(empty.columns.values)

In [ ]:
empty.shape

In [ ]:
X_empty = empty.drop('has_flaw', axis=1)
X_empty.shape

In [ ]:
y_empty = empty['has_flaw']
y_empty.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_empty, y_empty, random_state=0)

In [ ]:
print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))

In [ ]:
print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))

La celda a continuación tiene como objetivo estandarizar los valores de las características para que tengan media 0 y varianza 1. Hacemos esto pues de forma similar a cómo sucedía con SVM, las NNs son sensitivas al escalado de características. Para corroborar esto se sugiere primeramente no ejecutar la celda de estandarización y ver la performance que tiene el clasificador. Luego, ejecutar la misma y las que siguen a continuación para poder comparar la diferencia existente en la calidad predictiva del clasificador.

In [ ]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# fit only on training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test) 

### Descripción de los parámetros de la NN

**activation** es la función de activación: {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}, por defecto ‘relu’.

**alpha** es el parámetro de penalización del término de regularización introducido en la función a optimizar para evitar el sobre-ajuste. En la transparencia 44 de la clase 7, es referido como *lambda*.

**hidden_layer_sizes** es el número de capas ocultas y de neuronas en cada capa. Por defecto, 1 capa oculta de 100 neuronas *(100,)*. Si quisiéramos poner tres capas ocultas de 10 neuronas cada una, deberíamos especificar *(10,10,10)*.

**solver** es el método utilizado para realizar la optimización de pesos: {‘lbfgs’, ‘sgd’, ‘adam’}, por defecto ‘adam’.
* *lbfgs*: optimizador de la familia de los métodos "quasi-Newton".
* *sgd*: descenso del gradiente estocástico.
* *adam*: descenso del gradiente estocástico propuesto por Kingma, Diederik, and Jimmy Ba.

**learning_rate** constante positiva que nos permite moderar cuán pronunciada es la actualización de los pesos en cada paso: {‘constant’, ‘invscaling’, ‘adaptive’}, por defecto ‘constant’.
* *constant* es un valor constante dado por el parámetro ‘learning_rate_init’.
* *invscaling* decrementa gradualmente el learning rate en cada paso de tiempo ‘t’: effective_learning_rate = learning_rate_init / pow(t, power_t).
* *adaptive* mantiene el learning rate constante en ‘learning_rate_init’ mientras la función de pérdida en el entrenamiento siga disminuyendo.

**momentum** valor constante para actualizar el descenso del gradiente, por defecto ‘0.9’. Sólo usado cuando solver=‘sgd’.

In [ ]:
from sklearn.neural_network import MLPClassifier

nn = MLPClassifier(activation='logistic', solver='lbfgs', alpha=0.0001, hidden_layer_sizes=(5,), random_state=1)

nn.fit(X_train, y_train)     



In [ ]:
y_model = nn.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_model)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_model))

In [ ]:
target_names = ['no', 'yes']

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(y_test, y_model)

sns.heatmap(mat.T, square=True, annot=True, cbar=False, fmt="d", xticklabels=target_names, yticklabels=target_names)
plt.xlabel('valor verdadero')
plt.ylabel('valor predicho');



In [ ]:
nngs = MLPClassifier(activation='logistic', solver='sgd', alpha=0.0001, hidden_layer_sizes=(5,), random_state=1)

from sklearn.model_selection import GridSearchCV
parameters = {'momentum': [0.7,0.8,0.9], 'learning_rate_init': [0.001,0.01,0.1]}

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(nngs, parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    #print()
    #print("Grid scores on development set:")
    #print()
    #means = clf.cv_results_['mean_test_score']
    #stds = clf.cv_results_['std_test_score']
    #for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    #    print("%0.3f (+/-%0.03f) for %r"
    #          % (mean, std * 2, params))
    #print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()
